## Initial setup

### Check environment requirements

**IMPORTANT!** This notebook has special environment requirements. Makes sure to install Hail (https://hail.is/docs/0.2/index.html) and run on Google Dataproc clusters

### Load dependencies

In [ ]:
# Load libraries

# General use and data science packages
import os
import io
import json
import pandas as pd
import sys
import numpy as np

# Hail-specific packages
import hail as hl


### Start a Hail session

In [ ]:
# Initialize the Hail engine
hl.init(log = 'ces.log')

## Clinical-exome

### Load meta

In [ ]:
#load meta data to hail

ces_meta = hl.import_table(f'{bucket}/pf-exome-broswer/PDGENE_full_data_label.txt',
key='s',
impute=True)


In [ ]:
# find unique ancestry
labels=ces_meta.pop.collect()
ancestry = list(dict.fromkeys(labels))
ancestry

In [ ]:
# check ancestry count
print(f'Count for ancestry : {meta.aggregate(hl.agg.counter(ces_meta.pop))}') 


### import vcf

In [ ]:
# Run the import function on the GCS file path

mt = hl.import_vcf(f'{bucket}/pf-exome-broswer/all_chrs.vcf.gz',reference_genome='GRCh38', force_bgz=True)

mt.describe()

### prepare variant annotation table

In [ ]:
# split CSQ to array that we can choose
mt = mt.annotate_rows(csq_array = mt.info.CSQ.map(lambda x: x.split('\|'))[0])
#keep the field we want
# Find the index of 'SYMBOL'
consequence = CSQ.index("Consequence")
gene_id = CSQ.index("Gene")
transcript_id = CSQ.index("Feature")
gene_name = CSQ.index("SYMBOL")
hgvsc = CSQ.index("HGVSc")
hgvsp = CSQ.index("HGVSp")
cadd = CSQ.index("CADD_PHRED")

mt = mt.annotate_rows(consequence = mt.csq_array[consequence],
                                gene_id = mt.csq_array[gene_id],
                                transcript_id = mt.csq_array[transcript_id],
                                gene_name = mt.csq_array[gene_name],
                                hgvsc = mt.csq_array[hgvsc],
                                hgvsp = mt.csq_array[hgvsp],
                                cadd = mt.csq_array[cadd]
                               )
mt.describe()

In [ ]:
#update all the '' to be NaN
mt= mt.annotate_rows(hgvsp = hl.if_else(mt.hgvsp!="", mt.hgvsp, hl.missing('str')),
                     hgvsc = hl.if_else(mt.hgvsc!="", mt.hgvsc, hl.missing('str')),
                     gene_name = hl.if_else(mt.gene_name!="", mt.gene_name, hl.missing('str')),
                     cadd = hl.if_else(mt.cadd!="", mt.cadd, hl.missing('str'))         
                    )

In [ ]:
mt = mt.select_rows(variant_id = mt.rsid,
                    gene_id = mt.gene_id,
                    transcript_id = mt.transcript_id,
                    consequence = mt.consequence,
                    gene_name = mt.gene_name,
                    hgvsc = mt.hgvsc,
                    hgvsp = mt.hgvsp,
                    cadd = mt.cadd                                 
                    )

# keep just the row fields and filter out gene_id w/o gene_name 
# filter out intergenic_variant
# change cadd to float64 
ces_anno_ht = mt.rows()
ces_anno_ht = ces_anno_ht.filter((ces_anno_ht.gene_id != '') & (ces_anno_ht.consequence != 'intergenic_variant'))

# change cadd to float64 
ces_anno_ht = ces_anno_ht.annotate(cadd = hl.float(ces_anno_ht.cadd))

ces_anno_ht.show()

In [ ]:
ces_anno_ht = ces_anno_ht.annotate(hgvsp = ces_anno_ht.hgvsp.replace("%3D", "="))
ces_anno_ht = ces_anno_ht.annotate(variant_id = ces_anno_ht.variant_id.replace("_", ":"))

ces_anno_ht.filter(hl.is_defined(ces_anno_ht.hgvsp)).show()

### Write out variant annotation table

In [ ]:
# write out 
ces_anno_ht.write(f'{out_path}/browser_variant_annotation_table.ht', overwrite=True)

### Prep Variant results table

In [ ]:
mt = mt.annotate_cols(**ces_meta[mt.s])

mt.col.show(5)


In [ ]:
#remove CES that are dup of WGS
s_to_rm = hl.import_table(f'{bucket}/pf-exome-broswer/ces_to_remove_dup.txt',
key='s',
impute=True)

s_to_rm.describe()


In [ ]:
#remove CES that are dup of WGS
mt = mt.filter_cols(hl.is_defined(s_to_rm[mt.s]), keep=False)

In [ ]:
# get ancestry

lables=mt.pop.collect()
ancestry = list(dict.fromkeys(lables))
ancestry

In [ ]:
call_stats_expression = []

for group in ancestry:
    call_stats_expression.append(
        hl.struct(ancestry=group,
                  dataset= 'CES',
                  case=hl.agg.filter(
                        (mt.pop==group) & (mt.pheno=='Case'), 
                        hl.agg.call_stats(mt.GT, mt.alleles)), 
                  ctrl=hl.agg.filter(
                        (mt.pop==group) & (mt.pheno=='Control'), 
                        hl.agg.call_stats(mt.GT, mt.alleles)))
)
    
mt_freq = mt.annotate_rows(explode_data=call_stats_expression)

In [ ]:
# to explode

mt_freq = mt_freq.explode_rows('explode_data')
mt_freq = mt_freq.transmute_rows(**mt_freq.explode_data)


In [ ]:
#output to hail table
ht = mt_freq.rows()

ht.show()


In [ ]:
#filter out intergenic variants
ht= ht.filter(ht.consequence != 'intergenic_variant')

## keep the columns

ht = ht.select(
    variant_id = ht.variant_id,
    dataset = ht.dataset,
    ancestry = ht.ancestry,
    ac_case = ht.case.AC,
    an_case = ht.case.AN,
    af_case = ht.case.AF
)

ht.show(10)


In [ ]:
#write out
ht.write(BROWSER_VARIANT_RESULTS_TABLE, overwrite=True)


## WGS

### import vcf

In [ ]:
mt = hl.import_vcf(vcf_path,force_bgz=True,
                   reference_genome='GRCh38')


mt.describe()

### read meta

In [ ]:
# read meta

meta = hl.import_table(f'{bucket}/wgs_r10/r10_hail_sample_meta_table.txt',
key='s',
impute=True)

meta.show()

In [ ]:
# find unique ancestry
wgs_labels= meta.pop.collect()
wgs_ancestry = list(dict.fromkeys(wgs_labels))
wgs_ancestry

### prep variant annotation table

In [ ]:
ht = hl.import_table(annotation_table,missing='""')
ht.describe()

In [ ]:
#replace vep synonymous %3D
ht = ht.annotate(hgvsp = ht.hgvsp.replace("%3D", "="))
ht = ht.annotate(variant_id = ht.variant_id.replace("_", ":"))
# change cadd to float64 
ht = ht.annotate(cadd = hl.float(ht.cadd))


In [ ]:
# parse locus and alleles 
ht = ht.key_by(**hl.parse_variant(ht.variant_id,reference_genome='GRCh38'))

In [ ]:
# write out 
ht.write(f'{out_path}/browser_variant_annotation_table.ht', overwrite=True)

### prep freq table

In [ ]:
mt = mt.annotate_cols(**meta[mt.s])
mt.describe()

In [ ]:
call_stats_expression = []

for group in wgs_ancestry:
    call_stats_expression.append(
        hl.struct(ancestry=group,
                  dataset= 'WGS',
                  case=hl.agg.filter(
                        (mt.pop==group) & (mt.pheno=='Case') & (mt.keep==1), 
                        hl.agg.call_stats(mt.GT, mt.alleles)), 
                  other=hl.agg.filter(
                        (mt.pop==group) & (mt.pheno=='Other') & (mt.keep==1), 
                        hl.agg.call_stats(mt.GT, mt.alleles)), 
                  ctrl=hl.agg.filter(
                        (mt.pop==group) & (mt.pheno=='Control') & (mt.keep==1), 
                        hl.agg.call_stats(mt.GT, mt.alleles)))
)
    
wgs_mt_freq = mt.annotate_rows(explode_data=call_stats_expression)

# to explode
wgs_mt_freq = wgs_mt_freq.explode_rows('explode_data')
wgs_mt_freq = wgs_mt_freq.transmute_rows(**wgs_mt_freq.explode_data)


In [ ]:
#output to hail table
wgs_ht = wgs_mt_freq.rows()
wgs_ht = wgs_ht.annotate(rsid = wgs_ht.rsid.replace("_", ":"))
wgs_ht = wgs_ht.rename({'rsid' : 'variant_id'})
wgs_ht = wgs_ht.drop('info', 'qual', 'filters')

wgs_ht = wgs_ht.select(
    variant_id = wgs_ht.variant_id,
    dataset = wgs_ht.dataset,
    ancestry = wgs_ht.ancestry,
    ac_case = wgs_ht.case.AC,
    an_case = wgs_ht.case.AN,
    af_case = wgs_ht.case.AF,
    ac_ctrl = wgs_ht.ctrl.AC,
    an_ctrl = wgs_ht.ctrl.AN,
    af_ctrl = wgs_ht.ctrl.AF,
    ac_other = wgs_ht.other.AC,
    an_other = wgs_ht.other.AN,
    af_other = wgs_ht.other.AF
)

In [ ]:
#write out

wgs_ht.write(BROWSER_VARIANT_RESULTS_TABLE, overwrite=True)